In [461]:
import numpy as np
import pandas as pd
import folium
import matplotlib as mpl
import matplotlib.cm as cm

In [476]:
#nRowsRead = 1000
df = pd.read_csv('/Users/nikitaseleznev/Downloads/Rus_schools_final.csv', delimiter=',', encoding='1251')
df.head()

,Unnamed: 0,name,struct,addr,lat,lon
0,0,"Муниципальное общеобразовательное учреждение ""...",(Муниципальное образовательное учреждение),"420087, Республика Татарстан, г. Казань, ул.Ри...",55.76370,49.18172
1,1,"Муниципальная образовательная школа-интернат ""...",(Муниципальное образовательное учреждение),"420103, Республика Татарстан, г. Казань, ул.Че...",55.82440,49.12312
2,2,Муниципальное учреждение образования для детей...,(Муниципальное образовательное учреждение),"420100, Республика Татарстан, г. Казань, ул. Ю...",55.74462,49.20560
3,3,"Муниципальное общеобразовательное учреждение ""...",(Муниципальное образовательное учреждение),"420011, Республика Татарстан, г. Казань, Ферма-2",55.71716,49.16222
4,4,Специальное (коррекционное)образовательное учр...,(Государственное образовательное учреждение),"420036, Республика Татарстан, г. Казань, ул.Ти...",55.84911,49.07660


In [477]:
df.shape

(42581, 6)

In [478]:
df['lat'].isnull().mean(), df['lon'].isnull().mean()

(0.0, 0.0)

In [479]:
center = {'lat': 55.431177, 'lon': 37.544737}

In [480]:
from geopy import distance
df['distance_to_center'] = df.apply(lambda r: int(distance.distance((r['lat'], r['lon']), 
                                                           (center['lat'], center['lon'])).meters), axis=1)

In [481]:
schls_in_Podolsk = df[df['addr'].str.split(',').str[0].str[:3].str.contains('142') & 
                      (df['addr'].str.contains('Подольск') |
                       df['addr'].str.contains('Климовск') |
                      df['addr'].str.contains('Львовск'))]
schls_in_Podolsk = schls_in_Podolsk[schls_in_Podolsk['distance_to_center'] < 17000]
schls_in_Podolsk = schls_in_Podolsk[1:]

In [482]:
my_schl = schls_in_Podolsk[schls_in_Podolsk['addr'].str.contains('142118') & 
                 schls_in_Podolsk['addr'].str.contains('Свердлова')]
my_schl

,Unnamed: 0,name,struct,addr,lat,lon,distance_to_center
7544,7544,Муниципальное общеобразовательное учреждение Л...,(Муниципальное образовательное учреждение),"142118, Московская область, г. Подольск, ул Св...",55.41795,37.52831,1802


In [483]:
map_folium = folium.Map(location=(center['lat'], center['lon']),
                       tiles='Stamen Toner',
                       zoom_start =12)

In [484]:
Pod_ter_featGroup = folium.FeatureGroup(name='Podolsk territory')
Sch_ter_featGroup = folium.FeatureGroup(name='School grounds')

In [485]:
child_P = folium.GeoJson(data=open('/Users/nikitaseleznev/Downloads/Podolsk_json.json').read(), 
                         style_function=lambda x:{'fillColor': 'lightgray', 
                                                  'color': 'black', 
                                                  'weight': 2.5}, 
                        tooltip='Podolsk territory')
child_S = folium.GeoJson(data=open('/Users/nikitaseleznev/Downloads/school_polygon.geojson').read(), 
                         style_function=lambda x:{'fillColor': 'red', 'color': 'darkblue'}, 
                        tooltip='School grounds')

In [486]:
Pod_ter_featGroup.add_child(child_P, name='Podolsk_polygon')
Sch_ter_featGroup.add_child(child_S, name='School_polygon')
Pod_ter_featGroup.add_to(map_folium)
Sch_ter_featGroup.add_to(map_folium)

In [487]:
folium.LayerControl(collapsed=False).add_to(map_folium)

In [488]:
norm = mpl.colors.Normalize(vmin=0, vmax=5000)
cmap = cm.plasma
colormap = cm.ScalarMappable(norm=norm,cmap=cmap)
for ind, row in schls_in_Podolsk.iterrows():
    if ind != my_schl.index:
        folium.CircleMarker([row['lat'], row['lon']],
                           radius=7,
                           popup= str(row['distance_to_center']) + 'm from the center',
                           color=mpl.colors.rgb2hex((colormap.to_rgba(row['distance_to_center']))),
                           fill_color=mpl.colors.rgb2hex((colormap.to_rgba(row['distance_to_center']))),
                           fill=True,
                           fill_opacity=0.8).add_to(map_folium)
    else:
        folium.CircleMarker([row['lat'], row['lon']],
                           radius=15,
                           popup=str(row['distance_to_center']) + 'm from the center',
                           color=mpl.colors.rgb2hex((colormap.to_rgba(row['distance_to_center']))),
                           fill_color='k',
                           fill=True,
                           fill_opacity=0.15, 
                           tooltip='My school').add_to(map_folium)

#folium.features.ClickForMarker(popup=None).add_to(map_folium)
map_folium